# Imports

In [1]:
import os
import json
from pathlib import Path

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
from tensorflow.keras.regularizers import l2

In [6]:
from tensorflow.keras.utils import to_categorical

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from sentiment_analyser import Sentiment_Analyser

In [10]:
data_store=Path(os.getcwd()).parent.parent/"Data_Store"

# Load Data

In [11]:
data=pd.read_csv(
    os.path.join(
        data_store,
        "sentiment_analyser_prepared_data.csv"
    )
)

In [12]:
data

,cleaned_text,label
0,george kaitlyn a teacher at pine view high rec...,2
1,zimbabwe local businesses are grappling with a...,1
2,billionaire elon musk has donated to a super p...,2
3,a trade regulator on fridayannounced a suite o...,0
4,million households in the have solar panels on...,2
...,...,...
3495,qrg capital management increased its stake in ...,2
3496,qrg capital management bought a new position i...,2
3497,qrg capital management boosted its stake in ak...,2
3498,july according to the global biotechnology ma...,1


In [13]:
data_X=data["cleaned_text"]
data_Y=data["label"]

# Preparing Data

## Tokenize Data

In [14]:
tokenizer=Tokenizer()

In [15]:
tokenizer.fit_on_texts(data_X)

In [16]:
data_X=tokenizer.texts_to_sequences(data_X)

In [17]:
word_index=tokenizer.word_index

### Saving Tokenizer

In [18]:
tokenizer_json=tokenizer.to_json()

In [19]:
with open(os.path.join(data_store,"sentiment_analyser_tokenizer.json"),"w") as file:
    json.dump(tokenizer_json,file)

## Padding Texts

In [20]:
data_X=pad_sequences(data_X,maxlen=50,padding="post",truncating="post")

## One-Hot Encoding of Labels

In [21]:
data_Y=to_categorical(data_Y)

# Dataset Splitting

In [22]:
train_X,val_X,train_Y,val_Y=train_test_split(data_X,data_Y,test_size=0.2,random_state=42)

# Model Parameters

## LSTM Layers Config

In [23]:
lstm_layers_config=[
    {
        "units":128,
        "activation":"relu",
        "kernel_initializer":"he_uniform",
        "kernel_regularizer":None,
        "return_sequences":True
    },
    {
        "units":256,
        "activation":"relu",
        "kernel_initializer":"he_uniform",
        "kernel_regularizer":None,
        "return_sequences":True
    },
    {
        "units":64,
        "activation":"relu",
        "kernel_initializer":"he_uniform",
        "kernel_regularizer":l2(0.005),
        "return_sequences":True
    },
    {
        "units":16,
        "activation":"relu",
        "kernel_initializer":"he_uniform",
        "kernel_regularizer":None,
        "return_sequences":False
    }
]

## Dense Layers Config

In [24]:
dense_layers_config=[
    {
        "units":128,
        "activation":"relu",
        "kernel_initializer":"he_uniform",
        "kernel_regularizer":None
    },
    {
        "units":512,
        "activation":"relu",
        "kernel_initializer":"he_uniform",
        "kernel_regularizer":None
    },
    {
        "units":256,
        "activation":"relu",
        "kernel_initializer":"he_uniform",
        "kernel_regularizer":l2(0.007)
    },
    {
        "units":64,
        "activation":"relu",
        "kernel_initializer":"he_uniform",
        "kernel_regularizer":l2(0.007)
    },
    {
        "units":16,
        "activation":"relu",
        "kernel_initializer":"he_uniform",
        "kernel_regularizer":None
    },
    {
        "units":8,
        "activation":"relu",
        "kernel_initializer":"he_uniform",
        "kernel_regularizer":None
    },

    {
        "units":3,
        "activation":"softmax",
        "kernel_initializer":"glorot_uniform",
        "kernel_regularizer":None
    }
]

## Optimizer Config

In [25]:
optimizer_config={
    "learning_rate":0.001,
    "beta_1":0.96,
    "beta_2":0.98,
    "clipnorm":1.0
}

# Sentiment-Analyser

## Initialization

In [26]:
sentiment_analyser=Sentiment_Analyser(
     num_multi_lstm_units=5,
     lstm_layers_config=lstm_layers_config,
     dense_layers_config=dense_layers_config,
     optimizer_config=optimizer_config,
     model_loss_function="categorical_crossentropy",
     word_index=word_index,
     context_window=10,
     embedding_dim=100
)

## Build Model

In [27]:
sentiment_analyser.create_model()

[INFO] Embedding matrix generated
[WARNING] 3326 words embedding not found!!!
[INFO] Building Model


2025-08-30 01:24:31.895171: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


[INFO] Model Built
[INFO] Model Compiled
[INFO] Model Architecture: 


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 10, 100)   │  1,547,900 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 10, 100)   │  1,547,900 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 10, 100)   │  1,547,900 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 10, 100)   │  1,547,900 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 10, 100)   │  1,547,900 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_lstm__unit    │ (None, 16)        │    599,776 │ embedding[0][0]   │
│ (Multi_LSTM_Unit)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_lstm__unit_1  │ (None, 16)        │    599,776 │ embedding_1[0][0] │
│ (Multi_LSTM_Unit)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_lstm__unit_2  │ (None, 16)        │    599,776 │ embedding_2[0][0] │
│ (Multi_LSTM_Unit)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_lstm__unit_3  │ (None, 16)        │    599,776 │ embedding_3[0][0] │
│ (Multi_LSTM_Unit)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_lstm__unit_4  │ (None, 16)        │    599,776 │ embedding_4[0][0] │
│ (Multi_LSTM_Unit)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 80)        │          0 │ multi_lstm__unit… │
│ (Concatenate)       │                   │            │ multi_lstm__unit… │
│                     │                   │            │ multi_lstm__unit… │
│                     │                   │            │ multi_lstm__unit

 Total params: 10,963,775 (41.82 MB)

 Trainable params: 10,963,775 (41.82 MB)

 Non-trainable params: 0 (0.00 B)

## Train Model

In [28]:
batch_size=32
epochs=50

In [29]:
callback=EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=10,
    mode="min",
    restore_best_weights=True
)

In [30]:
sentiment_analyser.train_model(
    train_X=train_X,
    train_Y=train_Y,
    val_X=val_X,
    val_Y=val_Y,
    batch_size=batch_size,
    epochs=epochs,
    callback=callback
)

[INFO] Data Prepared for Training
[INFO] Model Training started
Epoch 1/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 134s 351ms/step - accuracy: 0.5917 - loss: 13.4221 - val_accuracy: 0.6829 - val_loss: 3.6483
Epoch 2/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 22s 248ms/step - accuracy: 0.7257 - loss: 2.6003 - val_accuracy: 0.7171 - val_loss: 1.2182
Epoch 3/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 30s 128ms/step - accuracy: 0.7598 - loss: 0.9962 - val_accuracy: 0.7029 - val_loss: 0.9251
Epoch 4/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 124ms/step - accuracy: 0.7622 - loss: 0.7566 - val_accuracy: 0.7186 - val_loss: 0.8582
Epoch 5/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 121ms/step - accuracy: 0.7958 - loss: 0.5979 - val_accuracy: 0.7057 - val_loss: 0.8523
Epoch 6/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 125ms/step - accuracy: 0.8322 - loss: 0.4949 - val_accuracy: 0.7529 - val_loss: 0.7494
Epoch 7/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 124ms/step - accuracy: 0.8942 - loss: 0.3707 - val_accuracy: 0.7514 - val_loss: 0.7985
Epoch 8/50
88/88 ━━━━━━━━━━━━━━

## Save Model

In [31]:
sentiment_analyser.save_model()

[INFO] Model saved at path: /home/spiralmonster/Projects/DeepDoc/Data_Store/sentiment_analyser_model.h5
[INFO] Model saved at path: /home/spiralmonster/Projects/DeepDoc/Data_Store/sentiment_analyser_model.keras


# END